In [7]:
import geopandas as gpd
import copy

import networkx as nx
import snman
from snman import osmnx_customized as oxc

# Set these paths according to your own setup
data_directory = 'C:/Users/lballo/polybox/Research/SNMan/SNMan Shared/data/'
inputs_path = data_directory + 'inputs/'
export_path = data_directory + 'outputs/'
process_path = data_directory + 'process/'

In [8]:
# =====================================================================================
# LOAD DATA
# =====================================================================================

print('Load street graph')
G = snman.io.load_street_graph(process_path + 'edges_all_attributes.gpkg', process_path + 'nodes_all_attributes.gpkg')

print('Load rebuilding regions')
# Polygons that define which streets will be reorganized
rebuilding_regions_gdf = snman.io.load_rebuilding_regions(
    inputs_path + 'rebuilding_regions/rebuilding_regions.gpkg'
)

Load street graph
Load rebuilding regions


In [9]:
# =====================================================================================
# GIVEN LANES
# =====================================================================================

if 1:
    print('Set given lanes')
    snman.set_given_lanes(G, bidirectional_for_dead_ends=False)

if 1:
    print('Create directed graph of given lanes')
    G_minimal_graph_input = snman.create_given_lanes_graph(G, hierarchies_to_remove={snman.hierarchy.HIGHWAY})

Set given lanes
Create directed graph of given lanes


In [10]:
# =====================================================================================
# REBUILD
# =====================================================================================

if 1:
    print('Rebuild regions')
    snman.owtop.rebuild_regions(G, rebuilding_regions_gdf, verbose=True)

Rebuild regions
1 {'2_local_road'}
Initialized graph has  36  nodes and  88  edges
Iteration  1
Iteration  2
Iteration  3
Iteration  4
Iteration  5
Iteration  6
Iteration  7
Iteration  8
Iteration  9
Iteration  10
Iteration  11
Iteration  12
Iteration  13
Iteration  14
Iteration  15
Iteration  16
Iteration  17
Iteration  18
Iteration  19
Iteration  20
Iteration  21
Iteration  22
Iteration  23
Iteration  24
Iteration  25
Iteration  26
Iteration  27
Iteration  28
Iteration  29
Iteration  30
Iteration  31
Iteration  32
Iteration  33
Iteration  34
Iteration  35
Iteration  36
Iteration  37
Iteration  38
Iteration  39
Iteration  40
Iteration  41
Iteration  42
Iteration  43
Iteration  44
Iteration  45
Iteration  46
Iteration  47
Iteration  48
Iteration  49
Iteration  50
Iteration  51
Iteration  52
Iteration  53
Iteration  54
Iteration  55
Iteration  56
Iteration  57
Iteration  58
Iteration  59
Iteration  60
Iteration  61
Iteration  62
Iteration  63
Iteration  64
Iteration  65
Iteration  66
It

In [11]:
# =====================================================================================
# EXPORT
# =====================================================================================

if 1:
    print('Export network without lanes')
    snman.export_streetgraph(G, export_path + 'edges_all_attributes.gpkg', export_path + 'nodes_all_attributes.gpkg')
    snman.export_streetgraph(G, export_path + 'edges.gpkg', export_path + 'nodes.gpkg',
        edge_columns=snman.constants.EXPORT_EDGE_COLUMNS
    )

if 0:
    print('Export OSM XML')
    snman.export_osm_xml(G, export_path + 'new_network.osm',{
        'highway', 'lanes', 'lanes:forward', 'lanes:backward', 'lanes:both_ways',
        'cycleway', 'cycleway:lane', 'cycleway:left', 'cycleway:left:lane', 'cycleway:right', 'cycleway:right:lane',
        'bus:lanes:backward', 'bus:lanes:forward', 'vehicle:lanes:backward', 'vehicle:lanes:forward',
        'maxspeed', 'oneway',
        '_connected_component'
    }, uv_tags=True, tag_all_nodes=True)

if 1:
    print('Export network with lanes')
    snman.export_streetgraph_with_lanes(G, 'ln_desc', export_path + 'edges_lanes.shp', scaling=2)
    snman.export_streetgraph_with_lanes(G, 'ln_desc_after', export_path + 'edges_lanes_after.shp', scaling=2)

Export network without lanes
Export network with lanes
